<a href="https://colab.research.google.com/github/tomonari-masada/course2021-stats2/blob/main/02_PyMC_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. PyMC入門（１）
* キャメロン・デビッドソン=ピロン(著), 玉木徹(訳)：『Pythonで体験するベイズ推論:PyMCによるMCMC入門』, 森北出版 (2017)
 * https://www.amazon.co.jp/dp/4627077912
* 今回の授業資料で参考にしたコード（上の本の公式のリポジトリから）
 * https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter1_Introduction/Ch1_Introduction_PyMC3.ipynb

## 2.0 インストール
* arviz関係のエラーが出たら、ランタイムを再起動して、そこから上のセルを実行し直す。

In [1]:
!pip install arviz==0.10

In [2]:
!pip install pymc3==3.8

## 2.1 はじめに

### 2.1.A PyMC3における確率変数

* とりあえずPyMC3をインポートする。
 * エラーが出たらランタイムを再起動するなどしてみる。

In [3]:
import pymc3 as pm

* PyMC3では、いきなり確率変数を作ることはできない！
 * パラメータが1の指数分布に従う確率変数を作ろうとして、下のセルを実行すると、エラーが出るはず。

In [4]:
lambda_ = pm.Exponential("poisson_param", 1)

TypeError: ignored

* 確率変数はいきなり作るのではなく、必ずモデルという文脈の中で作る。
 * モデルはあらかじめ空のインスタンスを作っておき、「with model:」というブロックを書くか、
 * いきなり「with pm.Model() as model:」と書くかの、いずれか。

In [5]:
model = pm.Model()
with model:
    lambda_ = pm.Exponential("poisson_param", 1)

In [6]:
with pm.Model() as model:
    lambda_ = pm.Exponential("poisson_param", 1)

* 下のモデルは、まず、平均パラメータが0で標準偏差パラメータが1の正規分布に従うunobservedな確率変数muを持っている。
 * この正規分布は、事前分布。
* そして、平均パラメータが確率変数muで標準偏差パラメータが1の正規分布に従うobservedな確率変数obsを持っている。
 * この正規分布は、データを直接モデリングする確率分布。
* さらに、観測データとして、正規乱数として生成した100個の数値を指定している。
---
* このベイズ的なモデルを数式で書くと・・・
$$ \mu \sim N(0, 1) $$
$$ x \sim N(\mu, 1) $$



In [7]:
import numpy as np

model = pm.Model()
with model:
    mu = pm.Normal("mu", mu=0, sigma=1)
    obs = pm.Normal("obs", mu=mu, sigma=1, observed=np.random.randn(100))

In [8]:
model.basic_RVs

[mu, obs]

In [9]:
model.free_RVs

[mu]

In [10]:
model.observed_RVs

[obs]

* 観測データを100個指定しているので、logpを計算することができる。
 * logpは、後で見るように、同時確率の対数（尤度の対数ではない）

In [11]:
model.logp({"mu": 0})

array(-142.30698916)

In [12]:
model.logp({mu: 0})

array(-142.30698916)

* モデルに属するそれぞれの確率変数は、適当な初期値を設定されている。
 * この初期値は、サンプリングの出発点として用いられる。

In [13]:
mu.tag.test_value

0.0

In [14]:
obs.tag.test_value

array([-0.34299566, -1.57579613,  1.71517655,  0.89709066, -0.00909118,
        0.95542039,  0.91907999,  0.39416902,  0.72182413, -0.02741638,
        0.40847715,  1.52460683,  0.70718402,  0.59635839, -0.55622266,
       -0.81089366, -1.21571075, -0.82206719, -1.52018009,  0.18107061,
       -0.0284776 , -0.06128338,  0.01669006, -0.11216392,  1.45085173,
       -0.92028141, -0.19304453,  0.95774958, -1.25881075, -0.27193824,
        0.53679621,  0.100623  ,  0.26745986, -0.97452024, -0.07042789,
       -0.67537728,  1.55122915,  0.00484454, -0.994258  , -0.13928114,
       -1.42999909,  0.67644212,  0.21670315, -0.15547204, -0.78475401,
        0.53225569, -3.19543303,  1.5370369 ,  1.65988444,  0.24052496,
        0.182978  ,  1.08508894, -1.02380637,  0.21299828,  0.31325545,
        0.77292977, -0.56311335, -0.92228738,  0.52874121, -1.83968657,
        0.18263411,  0.26028187,  1.05711495, -1.35556163,  0.56606363,
       -0.19656537,  1.34774775, -1.40209482, -0.71927715,  1.31

In [15]:
obs.observations

array([-0.34299566, -1.57579613,  1.71517655,  0.89709066, -0.00909118,
        0.95542039,  0.91907999,  0.39416902,  0.72182413, -0.02741638,
        0.40847715,  1.52460683,  0.70718402,  0.59635839, -0.55622266,
       -0.81089366, -1.21571075, -0.82206719, -1.52018009,  0.18107061,
       -0.0284776 , -0.06128338,  0.01669006, -0.11216392,  1.45085173,
       -0.92028141, -0.19304453,  0.95774958, -1.25881075, -0.27193824,
        0.53679621,  0.100623  ,  0.26745986, -0.97452024, -0.07042789,
       -0.67537728,  1.55122915,  0.00484454, -0.994258  , -0.13928114,
       -1.42999909,  0.67644212,  0.21670315, -0.15547204, -0.78475401,
        0.53225569, -3.19543303,  1.5370369 ,  1.65988444,  0.24052496,
        0.182978  ,  1.08508894, -1.02380637,  0.21299828,  0.31325545,
        0.77292977, -0.56311335, -0.92228738,  0.52874121, -1.83968657,
        0.18263411,  0.26028187,  1.05711495, -1.35556163,  0.56606363,
       -0.19656537,  1.34774775, -1.40209482, -0.71927715,  1.31

* 下のセルの計算で、上の計算と同じ答えを得ることができているのは、なぜか。説明してみよう。

In [16]:
import numpy as np
from scipy.stats import norm

print(np.log(norm().pdf(obs.observations)).sum() + np.log(norm().pdf(0)))

-142.30698915626672


* 初期値は、以下のようにして手動で設定することもできる。

In [17]:
with model:
    parameter = pm.Exponential("poisson_param", 1.0, testval=0.5)

print("parameter.tag.test_value =", parameter.tag.test_value)

parameter.tag.test_value = 0.5


* 同じ分布に従う複数の確率変数を一挙に作ることもできる。

In [18]:
N=100
with pm.Model() as model:
    betas = pm.Uniform("betas", 0, 1, shape=N)

In [19]:
betas.tag.test_value

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])